In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle as p
cv2.__version__

'3.4.5'

In [2]:
#paths 

folder = './models'
model_path = os.path.join(folder,'ssd.caffemodel')
proto_path = os.path.join(folder,'deploy.prototxt')

In [3]:
dnet = cv2.dnn.readNetFromCaffe(proto_path,model_path)

In [4]:
#face net model
enet = cv2.dnn.readNetFromTorch('./models/openface_nn4.small2.v1.t7')

In [27]:
img = cv2.imread('./people/unknown/image100.jpg')
print(img.shape)
blob_img = cv2.dnn.blobFromImage(img,1.0,size = (300,300),mean = (123.68,116.78,103.94),swapRB = False,crop = False)
blob_img.shape

h,w,_ = img.shape

dnet.setInput(blob_img)
r = dnet.forward()


(750, 562, 3)


In [17]:
r[0][0][0:3]

array([[0.        , 1.        , 0.9999877 , 0.22753325, 0.19497436,
        0.77354085, 0.8144837 ],
       [0.        , 1.        , 0.9991992 , 0.56312394, 0.5480316 ,
        1.1001501 , 1.1428251 ],
       [0.        , 1.        , 0.1046887 , 4.1802025 , 4.0172906 ,
        4.8378754 , 4.9839787 ]], dtype=float32)

In [33]:
r[0][0][3][2] = 1

In [50]:
import cv2
c =  filter_op(r)
print(c)    
    
if len(c) > 1:
   
    c = nms(c)
    
    for i in c:
    
        cv2.rectangle(img,(i[0],i[1]),(i[2],i[3]),(255,255,255),5)
        print(i)
cv2.imshow("dasd",img)
cv2.waitKey(0)
cv2.destroyAllWindows()
    

[array([127, 146, 434, 610]), array([316, 411, 618, 857]), array([  87, 3012,  462, 3737])]
[127 146 434 610]
[0 0 0 0]
[  87 3012  462 3737]


In [44]:
cv2.rectangle(img,(87,3012),(462,3737),(0,0,255),5)
    
cv2.imshow("dasd",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [46]:
def nms(box,th = 0.1):   
    
    for p in range(len(box)):
        
        box1 = box[p]
    
        if p == len(box) -1:
#             print('stopping')
            break
    
        for j in range(1,len(box)):
        
            box2 = box[j]
            result = iou2(box1,box2)
#             print('iou',result)
            if result != 0:
                box[j] = np.array([0,0,0,0])
#                 print(j)
    return box


def iou2(bbox1,bbox2):
    #content of box : x,y,width,height
    #formula  intersection/union
    
    
    x1,y1,w1,h1 = bbox1
    x2,y2,w2,h2 = bbox2
    
#     print(x1,y1,w1,h1)
  
    
    x3 = np.max((x1,x2))
    y3 = np.max((y1,y2))
    w3 = np.min((x1+w1,x2+w2)) - x3
    h3 = np.min((y1+h1,y2+h2)) - y3
    
    #print('w3',w3)
    #print('h3',h3)
    
    if w3<=0 or h3 <=0:
        return 0
    
    iint = w3*h3
    #print(iint)
    u = (w1*h1 + w2*h2) - iint
    #print("area1",w1*h1)
    #print("area2",w2*h2)
    
    return iint/u
 
# print(iou2(box[0],box[1]))
# box



In [18]:

def filter_op(r,conf = 0.9):
    
    
    box = []
    
    #filtering out using confidence levels
    for i in r[0,0,:]:

        if i[2] > conf:
            b = i[3:] * np.array([w,h,w,h])
            box.append(b.astype('int32'))
        
    return box    


In [7]:
def process(res,conf = 0.9,th = 0.1):
    
    
    rect = filter_op(res,conf)
#     print(len(rect))
#     if len(rect)>1:
#         print('perfroming nms')
#         n_rect = nms(rect,th)
#         return [x for x in n_rect if x.all() != 0]
#     else:
    return rect

In [13]:
def get_embedding(img,r):
    
    face = img[r[1]:r[3],r[0]:r[2]]
#     print(face.shape)
    if face.shape[0] == 0:
        return np.zeros(128)
    
    
#     print(face.shape)
    face_blob = cv2.dnn.blobFromImage(face,1/255,(96,96),swapRB = True)
#     print(face_blob.shape)
    enet.setInput(face_blob)
    return enet.forward()
    
    

In [8]:
# emb = []
# name = []
# for path in os.listdir('./people'):
    
#     print(path)
#     for img in os.listdir('./people/'+path):
        
        
#         img_path = './people/'+path+'/'+img
#         print(img_path)
#         i = cv2.imread(img_path)
        
        
#         blob_img = cv2.dnn.blobFromImage(i,1.0,size = (300,300),mean = (123.68,116.78,103.94),swapRB = False,crop = False)
#         dnet.setInput(blob_img)
#         result = dnet.forward()
# #         print(result.shape)
#         res = process(result,conf = 0)
#         print('no of detections:',len(res))
# #         plt.figure()
# #         plt.imshow(i[res[0][1]:res[0][3],res[0][0]:res[0][2]])
# #         plt.show()
#         emb.append(get_embedding(i,res[0]))
#         name.append(path)
        
        
# print('no of embeddigs and names',len(emb),len(name))    
# with open('./pickle_files/'+'emb.pickle','wb') as f:
#     p.dump({'emb':emb,'name':name},f)
        
    

In [9]:
#encoding labels 

from sklearn.preprocessing import LabelEncoder

with open('./pickle_files/emb.pickle','rb') as f:
    data = p.load(f)
    
label  = LabelEncoder()
l = label.fit_transform(data['name'])
print(l)
label.inverse_transform(l)


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]


array(['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown',
       'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown',
       'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown',
       'unknown', 'unknown', 'clive', 'clive', 'clive', 'clive', 'clive',
       'clive', 'clive', 'clive', 'clive', 'clive', 'clive', 'clive',
       'clive', 'clive', 'clive', 'clive', 'clive', 'clive', 'clive',
       'clive'], dtype='<U7')

In [10]:
# train_test_split?
# ytrain

In [22]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

x = np.squeeze(np.array(data['emb']))

xtrain,xtest,ytrain,ytest = train_test_split(x,l,test_size = 0.2,shuffle = True)

print(xtrain.shape,xtest.shape)
clf = SVC(C = 70.0,gamma = 0.001,probability = True)

clf.fit(xtrain,ytrain)
# np.squeeze(np.array(data['emb'])).shape,l.shape

(32, 128) (8, 128)


SVC(C=70.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [21]:
import cv2
#geqttiing training images
cam = cv2.VideoCapture(0)
while 1 :

    _,frame = cam.read()
    
    blob_img = cv2.dnn.blobFromImage(frame,1.0,size = (300,300),mean = (123.68,116.78,103.94),swapRB = False,crop = False)
    h,w,_ = frame.shape
    dnet.setInput(blob_img)
    result = dnet.forward()
#     print(result.shape)
    res = process(result,conf = 0.9)
#     print('no of detections:',len(res))
    
    if len(res) > 0:
        vec  = get_embedding(frame,res[0])
    
#         name = decode(vec)

#         name = label.inverse_transform(clf.predict(vec))[0]
        if vec.all() != 0:     
    
            pred = clf.predict_proba(vec)[0]
            j = np.argmax(pred)
#           print(label.classes_[j])
            score = pred[j]
            name =  label.classes_[j]
            if score < 0.5:
                name  = '----'



    else:
        print('no detction')
    
    frame = cv2.flip(frame,1)
    
    for o in res:
        cv2.rectangle(frame,(o[0],o[1]),(o[2],o[3]),(255,0,0),5)
        cv2.putText(frame,name+':'+str(score),(o[0],o[1]),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1,cv2.LINE_AA)

    
    cv2.imshow('img',frame)
    

    
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
cam.release()    
cv2.destroyAllWindows()   
        
    
    



no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction


In [14]:

cv2.flip?